In [26]:
import urllib.request
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
width = 13
vstat = "http://amk030.imces.ru/meteodata/AMK_030_BIN/2019/10_2019/10010008.19B"
page = urllib.request.urlopen(vstat)
content = page.read()

In [72]:
data_dopolnitelniy = np.frombuffer(content[:17], dtype=np.dtype('i1'))
data_dopolnitelniy = np.frombuffer(data_dopolnitelniy[:16], dtype=np.dtype('i2'))
date_temp = data_dopolnitelniy[:-1]
date_time = datetime(*date_temp).strftime("%d.%m.%Y %H:%M:%S.%f")
numpy_data_polojitelnie = np.frombuffer(content, dtype=np.dtype('B'))[17:-14]
numpy_data_polojitelnie = numpy_data_polojitelnie.reshape(int(len(numpy_data_polojitelnie) / width), width)
numpy_data_polojitelnie  = np.delete(numpy_data_polojitelnie,12,1)
numpy_data_polojitelnie = np.frombuffer(numpy_data_polojitelnie, dtype=np.dtype('i2'))
numpy_data_polojitelnie = numpy_data_polojitelnie.reshape(int(len(numpy_data_polojitelnie)/6),6)
row = numpy_data_polojitelnie.shape[0]
column = numpy_data_polojitelnie.shape[1] + 1
zero = np.zeros((row,1))
numpy_data_polojitelnie = np.append(numpy_data_polojitelnie,zero,axis=1) 
data_frame1 = pd.DataFrame(numpy_data_polojitelnie)
data_frame1.head()

,0,1,2,3,4,5,6
0,233.0,-184.0,-126.0,115.0,7437.0,9343.0,0.0
1,242.0,-188.0,-122.0,102.0,7438.0,9344.0,0.0
2,232.0,-178.0,-103.0,81.0,7437.0,9343.0,0.0
3,237.0,-172.0,-105.0,71.0,7437.0,9344.0,0.0
4,237.0,-172.0,-101.0,58.0,7437.0,9344.0,0.0


In [73]:
for i in range(len(numpy_data_polojitelnie)):
#     numpy_data_polojitelnie[i,0] = numpy_data_polojitelnie[i,0] // 100
    numpy_data_polojitelnie[i,1] = numpy_data_polojitelnie[i,1] / 100
    numpy_data_polojitelnie[i,2] = numpy_data_polojitelnie[i,2] / 100
    numpy_data_polojitelnie[i,3] = numpy_data_polojitelnie[i,3] / 100
    numpy_data_polojitelnie[i,4] = numpy_data_polojitelnie[i,4] / 10
    numpy_data_polojitelnie[i,5] = numpy_data_polojitelnie[i,5] / 10
    numpy_data_polojitelnie[i,6] = numpy_data_polojitelnie[i,6] / 100

dft = pd.DataFrame((numpy_data_polojitelnie), columns=[
    'A','A','A','A','A','A','A'],index=pd.date_range(
    date_time, periods=len(numpy_data_polojitelnie), freq='12.5ms'))
dft.rename_axis("limbs", axis="columns")
dft.columns.name = 'date and time'
dft.head()

date and time,A,A,A,A,A,A,A
2019-01-10 00:08:14.000343,233.0,-1.84,-1.26,1.15,743.7,934.3,0.0
2019-01-10 00:08:14.012843,242.0,-1.88,-1.22,1.02,743.8,934.4,0.0
2019-01-10 00:08:14.025343,232.0,-1.78,-1.03,0.81,743.7,934.3,0.0
2019-01-10 00:08:14.037843,237.0,-1.72,-1.05,0.71,743.7,934.4,0.0
2019-01-10 00:08:14.050343,237.0,-1.72,-1.01,0.58,743.7,934.4,0.0


In [76]:
from datetime import datetime
import pandas as pd
datelist = pd.date_range(date_time, periods=len(numpy_data_polojitelnie), freq='12.5ms').to_pydatetime()
# dt64 = [np.datetime64(x) for x in datelist] 
# dt64 = [int(dt64[x].item().strftime('%Y%m%d%H%M%S%f')) for x in range(len(dt64))]
pd.DataFrame({'Time': datelist, '0': numpy_data_polojitelnie[:,0], 'Temp': numpy_data_polojitelnie[:,1],
              '3': numpy_data_polojitelnie[:,2], '4': numpy_data_polojitelnie[:,3],
              '5': numpy_data_polojitelnie[:,4], '6': numpy_data_polojitelnie[:,5],
              '7': numpy_data_polojitelnie[:,6]})

,Time,0,Temp,3,4,5,6,7
0,2019-01-10 00:08:14.000343,233.0,-1.84,-1.26,1.15,743.7,934.3,0.0
1,2019-01-10 00:08:14.012843,242.0,-1.88,-1.22,1.02,743.8,934.4,0.0
2,2019-01-10 00:08:14.025343,232.0,-1.78,-1.03,0.81,743.7,934.3,0.0
3,2019-01-10 00:08:14.037843,237.0,-1.72,-1.05,0.71,743.7,934.4,0.0
4,2019-01-10 00:08:14.050343,237.0,-1.72,-1.01,0.58,743.7,934.4,0.0
5,2019-01-10 00:08:14.062843,237.0,-1.64,-1.11,0.67,743.7,934.4,0.0
6,2019-01-10 00:08:14.075343,234.0,-1.68,-1.16,0.67,743.7,934.3,0.0
7,2019-01-10 00:08:14.087843,237.0,-1.75,-1.15,0.60,743.7,934.3,0.0
8,2019-01-10 00:08:14.100343,240.0,-1.71,-1.20,0.65,743.7,934.2,0.0
9,2019-01-10 00:08:14.112843,240.0,-1.79,-1.19,0.69,743.7,934.2,0.0


In [ ]:
import pandas as pd
distance_array = []
for x in range(1,2):
    a=str(x) + ".csv"
    df = pd.read_csv(a, error_bad_lines=False, nrows=2)
    for index, row in df.iterrows():
        distance_array.append(x)
        print(index,row)
# print(sum(distance_array)/len(distance_array))